In [4]:
pip install matplotlib pandas scikit-learn

  Using cached matplotlib-3.7.2-cp310-cp310-macosx_11_0_arm64.whl (7.3 MB)
  Using cached pandas-2.0.3-cp310-cp310-macosx_11_0_arm64.whl (10.8 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 6.2 MB/s eta 0:00:0000:0100:01m
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Using cached Pillow-10.0.0-cp310-cp310-macosx_11_0_arm64.whl (3.1 MB)
  Using cached pyparsing-3.0.9-py3-none-any.whl (98 kB)
  Using cached contourpy-1.1.0-cp310-cp310-macosx_11_0_arm64.whl (229 kB)
  Using cached kiwisolver-1.4.4-cp310-cp310-macosx_11_0_arm64.whl (63 kB)
  Using cached pytz-2023.3-py2.py3-none-any.whl (502 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 7.9 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip available: 22.3.1 -> 23.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [43]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

This data is obtained by jigsaw toxic comment classification challenge....

In today's world uncensored and unlimited access to the internet is some thing that is quite common and hence such AI models help us to decrease hate, toxicity and more negativuty from the internet upto some extent. I am using the taining data from Kaggle, here is the link :
https://www.kaggle.com/competitions/jigsaw-toxic-comment-classification-challenge/data

This data is also available in the repositary if you want to take a look at it.

In [44]:
df = pd.read_csv('data/train.csv')

In [45]:
df.iloc[3]['comment_text']

'"\nMore\nI can\'t make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of ""types of accidents""  -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It\'s listed in the relevant form eg Wikipedia:Good_article_nominations#Transport  "'

The below are the type of outputs and parameters the comment classifier is going to be measuring by, usually most of the toxic comments can be classified as one of these. 

In [46]:
df[df.columns[2:]].iloc[3]

toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
Name: 3, dtype: int64

In [47]:
from tensorflow.keras.layers import TextVectorization

In [48]:
X = df['comment_text']
Y = df[df.columns[2:]].values

In [49]:
MAX_FEATURES = 2000000

Text vectorization is a preprocessing step commonly used in NLP to convert raw text data into numerical vectors that can be fed into machine learning models.
max_tokens-maximum number of unique tokens is considered(it is defined as MAX_FEATURES, in the previous step)
output_sequence_length-  maximum length of the vectorized sequences


In [50]:
vectorizer = TextVectorization(max_tokens=MAX_FEATURES,
                               output_sequence_length = 1800,
                               output_mode = 'int')

The 'adapt' method will tokenize the text and convert it into integer sequences based on the specified parameters.

In [51]:
vectorizer.adapt(X.values)

In [ ]:
vectorizer.get_vocabulary()

get_vocabulary- to retrieve the vocabulary learned by the text vectorization layer
The vocabulary list will be ordered based on the frequency of each token in the input data, with more frequent tokens appearing first.

In [53]:
vectorized_text = vectorizer(X.values)

In [54]:
vectorized_text

<tf.Tensor: shape=(159571, 1800), dtype=int64, numpy=
array([[   645,     76,      2, ...,      0,      0,      0],
       [219427,     54,   2489, ...,      0,      0,      0],
       [   425,    441,     70, ...,      0,      0,      0],
       ...,
       [ 32445,   7392,    383, ...,      0,      0,      0],
       [     5,     12,    534, ...,      0,      0,      0],
       [     5,      8,    130, ...,      0,      0,      0]])>

In [55]:
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, Y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)

In [56]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [57]:
train_generator = train.as_numpy_iterator()

In [58]:
train_generator.next()

(array([[   77,  1067,    16, ...,     0,     0,     0],
        [    2, 11285, 22121, ...,     0,     0,     0],
        [ 3899,     7,    20, ...,     0,     0,     0],
        ...,
        [    8,    69,     2, ...,     0,     0,     0],
        [   94,    13,     2, ...,     0,     0,     0],
        [    2,  4216,     4, ...,     0,     0,     0]]),
 array([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 1, 0, 1, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]]))

In [61]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout,Bidirectional,Dense,Embedding

Deep Learning components - 
Sequential method was chosed because it is fast and easy
I needed to import layers so I imported them in the below test

Creating Embedding layer- works like a personality test for a word.
After instatiating the sequentional layers, I am adding additional layers

In [62]:
model = Sequential()
model.add(Embedding(MAX_FEATURES+1,32))
model.add(Bidirectional(LSTM(32,activation='tanh')))
model.add(Dense(128,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(6,activation='sigmoid'))

Why 'tanh' - It is used in order to meet the gpu acceleration required by LSTM(it is standard by tensorflow )
What is Bidirectional? It can be helpful to pass information from both direction, and this is what bidirectional does. Trying to analyze words from a single direction may result in a few mistakes thus modifying the output. 

In the final layer, it has 6 units, representing 6 different values of the output. Sigmoid is used to get the output between 0 and 1

In [63]:
model.compile(loss = 'BinaryCrossentropy', optimizer='Adam')

In [64]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          64000032  
                                                                 
 bidirectional (Bidirection  (None, 64)                16640     
 al)                                                             
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                      

I can see 3 layers plus last output layer


In the next cell, the training took almost    mins. 

In [65]:
history = model.fit(train, epochs=1, validation_data=val)

6981/6981 [==============================] - 8243s 1s/step - loss: 0.0632 - val_loss: 0.0458


I can easily understand the history of the model by using history() function.

It is also suggested to use a greater number as epochs(maybe 10?) 

It is also easy to plot it on the graph

In [69]:
history.history

{'loss': [0.06322059035301208], 'val_loss': [0.04584613814949989]}

In [67]:
from matplotlib import pyplot as plt

In [ ]:
plt.figure(figsize=(10,5))
pd.DataFrame(history.history).plot()
plt.show()

In [70]:
batch = test.as_numpy_iterator().next()

In [79]:
batch_X, batch_y=test.as_numpy_iterator().next()

In [75]:
input_text = vectorizer('Your music is horrible! I am going to burn your records!')

In [76]:
model.predict(np.expand_dims(input_text, 0))

1/1 [==============================] - 0s 79ms/step


array([[0.94562286, 0.01572322, 0.6060938 , 0.01435961, 0.5110397 ,
        0.05518728]], dtype=float32)

In [ ]:
(model.predict(batch_X)>0.5).astype(int)

In [85]:
res = model.predict(batch_X)

1/1 [==============================] - 0s 94ms/step


In [80]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [81]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [91]:
for batch in test.as_numpy_iterator():
    X_true, y_true = batch
    yhat = model.predict(X_true)

    y_true = y_true.flatten()
    yhat = yhat.flatten()

    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

1/1 [==============================] - 0s 90ms/step


In [92]:
print(f'Precision: {pre.result().numpy()},Recall: {re.result().numpy()},Accuracy: {acc.result().numpy()}')

Precision: 0.8366690278053284,Recall: 0.6760034561157227,Accuracy: 0.47693079710006714


To improve, make the epoch larger. 

Everytime the number reaches closer to one, the model is better!!!

In [97]:
model.save('toxicity.h5')

/Users/apoorvav/.pyenv/versions/3.10.10/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
